# TrackNet Implementation

## Setup

### Imports

In [1]:
import os
import glob
import queue
import cv2 as cv
import numpy as np
import pandas as pd
import albumentations
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from turbojpeg import TurboJPEG
from PIL import Image,ImageDraw

from scipy import ndimage, misc
from scipy.signal import argrelextrema
from scipy.signal import savgol_filter

import torch
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader

### GPU

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

print('Device: ' + str(device))
if use_cuda:
    print('GPU: ' + str(torch.cuda.get_device_name(0)))

# Turns on cuDNN Autotuner
torch.backends.cudnn.benchmark = True

Device: cuda:0
GPU: NVIDIA GeForce RTX 2080 Ti


### TrackNet Params

In [3]:
# Folder paths
dataset_base_path = "./Dataset"
savePath_base = "./Trained_Models"
outputPath = "./Results"

# Training parameters
num_epochs = 15
learning_rate = 1e-4
eps = 1e-7

# TrackNet width and height
TN_width=640
TN_height=360

## Dataset

In [4]:
transform = albumentations.Compose([
  albumentations.Resize(height=TN_height, width=TN_width, interpolation=1, always_apply=True, p=1),
])

def getOutputArr(path,outClasses=256,w=TN_width,h=TN_height):
  seg_labels = np.zeros((h, w, outClasses))
  img = cv.imread(path, 1)
  img = cv.resize(img, (w, h))
  img = img[:, : , 0]
  for c in range(outClasses):
    seg_labels[:, :, c] = (img==c).astype(int)

  seg_labels = np.reshape(seg_labels, (w*h, outClasses))
  return seg_labels

In [5]:
class TN_Dataset(Dataset):
  def __init__(self, split):
    self.window_paths = []
    self.mask_paths = []
    self.jpeg_reader = TurboJPEG()
    
    game_list = os.listdir(f"{dataset_base_path}/{split}")
    # Iterate through each game
    for game in game_list:
      if game == "groundtruth":
        continue
      if int(game) > 10:
        continue
      game_dir = f"{dataset_base_path}/{split}/{game}"
      clips = os.listdir(game_dir)
      for clip in clips:
        # Read and store label information
        clip_dir = f"{game_dir}/{clip}"
        annotation_path = f"{clip_dir}/Label.csv"
        label = pd.read_csv(annotation_path)
        img_paths = np.asarray(label.iloc[:, 0])

        for frame_no, frame in enumerate(img_paths):
          # Skip frames not in this dataset
          if frame_no == 0 or frame_no == len(img_paths) - 1:
            continue
          window_path = []
          window_path.append(f"{clip_dir}/{img_paths[frame_no-1]}")
          window_path.append(f"{clip_dir}/{img_paths[frame_no]}")
          window_path.append(f"{clip_dir}/{img_paths[frame_no+1]}")
          self.window_paths.append(window_path)

          mask_path = f"{dataset_base_path}/{split}/groundtruth/{game}/{clip}/{frame.split('.')[0]}.png"
          self.mask_paths.append(mask_path)

  def loader(self, path):
    in_file = open(path, 'rb')
    image = self.jpeg_reader.decode(in_file.read(), 0)
    image = transform(**{"image": image})
    image = image['image']
    return image

  def __getitem__(self, index):
    img_1 = self.loader(self.window_paths[index][0])
    img_2 = self.loader(self.window_paths[index][1])
    img_3 = self.loader(self.window_paths[index][2])
    img_window = np.concatenate((img_1, img_2, img_3),axis=2)
    img_window = np.rollaxis(img_window, 2, 0)
    img_window = torch.from_numpy(img_window)
    
    label = getOutputArr(self.mask_paths[index])
    centre_img = cv.imread(self.window_paths[index][1])
    centre_img = cv.resize(centre_img, (TN_width, TN_height))
    centre_img.astype(np.float32)
    
    return img_window, label, centre_img

  def __len__(self):
    return len(self.window_paths)

In [ ]:
train_batch = 2
val_batch = 2
test_batch = 1
num_workers = 8

print("Loading Train...")
train_loader = DataLoader(TN_Dataset(split="train"), batch_size=train_batch,shuffle=True,num_workers=num_workers,pin_memory=True,drop_last=True)
print("Loading Test...")
test_loader = DataLoader(TN_Dataset(split="test"), batch_size=test_batch,shuffle=False,num_workers=num_workers)
print("Loading Validation...")
val_loader = DataLoader(TN_Dataset(split="val"), batch_size=val_batch,shuffle=False,num_workers=num_workers,pin_memory=True,drop_last=True)
print("Dataset total batches:")
print(f"Training : {len(train_loader)}")
print(f"Validation : {len(val_loader)}")
print(f"Test : {len(test_loader)}")

## TrackNet

In [7]:
class ConvBlock2(nn.Module):
  def __init__(self,in_channels,out_channels):
    super(ConvBlock2, self).__init__() 
    self.block = nn.Sequential(
      nn.Conv2d(in_channels = in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),          
      nn.ReLU(),
      nn.BatchNorm2d(out_channels),
      nn.Conv2d(in_channels = out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1), 
      nn.ReLU(),
      nn.BatchNorm2d(out_channels),
    )

  def forward(self, x): 
    return self.block(x)

class ConvBlock3(nn.Module):
  def __init__(self,in_channels,out_channels):
    super(ConvBlock3, self).__init__() 
    self.block = nn.Sequential(
      nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1), 
      nn.ReLU(),
      nn.BatchNorm2d(out_channels),
      nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1), 
      nn.ReLU(),
      nn.BatchNorm2d(out_channels),
      nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),          
      nn.ReLU(),
      nn.BatchNorm2d(out_channels),
    )

  def forward(self, x): 
    return self.block(x)

class TrackNet(nn.Module):
  def __init__(self,inchannel=3):
    super(TrackNet, self).__init__()
    self.down_sample = nn.Sequential(
      ConvBlock2(in_channels=inchannel, out_channels=64),
      nn.MaxPool2d(kernel_size=2, stride=2),
      ConvBlock2(in_channels=64, out_channels=128),
      nn.MaxPool2d(kernel_size=2, stride=2),
      ConvBlock3(in_channels=128, out_channels=256),
      nn.MaxPool2d(kernel_size=2, stride=2),
      ConvBlock3(in_channels=256, out_channels=512),
    )

    self.up_sample = nn.Sequential(
        nn.Upsample(scale_factor=2),
        ConvBlock3(in_channels=512, out_channels=256),
        nn.Upsample(scale_factor=2),
        ConvBlock2(in_channels=256, out_channels=128),
        nn.Upsample(scale_factor=2),
        ConvBlock2(in_channels=128, out_channels=64),
        nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(256),
    )

    self.softmax = nn.Softmax(dim=2)

  def forward(self,x):
    out = self.down_sample(x)
    out = self.up_sample(out)
    out_shape = out.shape
    out = out.reshape((out_shape[0],-1,out_shape[2]*out_shape[3]))
    out = out.permute(0,2,1)
    out = self.softmax(out)
    return out

## Train Model

In [54]:
######## Model and optimizer ########
model_saved = True
model = TrackNet(inchannel=9).to(device)
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)
scaler = GradScaler()
colors = [(i, i, i) for i in range(0, 256)]
train_loss_log = []
val_loss_log = []
model.train()
model.zero_grad()

In [ ]:
if (model_saved):
  savePath = f"{savePath_base}/New_loader/TrackNet_14.pth"
  print("Loading model from path: ")
  print(savePath)
  checkpoint = torch.load(savePath)
  model.load_state_dict(checkpoint['model'])
  optimizer.load_state_dict(checkpoint['optimizer'])
  start_epoch = checkpoint['epoch']
  train_loss_log = checkpoint['train_loss_log']
  val_loss_log = checkpoint['val_loss_log']
  print(f"Load epoch {start_epoch} successful")
else:
  start_epoch = 0
  print("No model to load, start training at epoch 0")

print('START TRAINING ...')
for epoch in range(start_epoch+1, num_epochs + 1):
  model.train()
  batch_num = len(train_loader)
  train_loss = 0
  train_loss_total = 0

  with tqdm(train_loader, unit="batch") as tepoch:
    for data_batch in tepoch:
      tepoch.set_description(f"Train Epoch {epoch}")
      # Read in train batch
      img_window, label, _ = data_batch
      window_batch = torch.as_tensor(img_window, dtype=torch.float32).to(device)
      label_tensor = torch.as_tensor(label, dtype=torch.float32).to(device)
      
      # Forward step and calculate loss
      with autocast():
        output = model(window_batch)
        train_loss = -torch.sum(label_tensor.mul(torch.log(output+eps)))/(output.shape[0]*output.shape[1]*output.shape[2])

      # Backward step
      optimizer.zero_grad(set_to_none=True)
      scaler.scale(train_loss).backward()
      scaler.step(optimizer)
      scaler.update()
      loss = train_loss.detach().cpu().numpy()
      train_loss_total += loss
      tepoch.set_postfix(loss=loss)

  # Log training losses
  tqdm.write(f"Train\t epoch: {epoch}/{num_epochs}\t loss: {train_loss_total/batch_num}")
  train_loss_log.append(train_loss_total/batch_num)

  # Validation dataset
  with torch.no_grad():   
    model.eval()
    batch_num = len(val_loader)
    val_loss = 0
    val_loss_total = 0

    with tqdm(val_loader, unit="batch") as vepoch:
      for data_batch in vepoch:
        vepoch.set_description(f"Validation Epoch {epoch}")
        # Read in val batch
        img_window, label, _ = data_batch
        window_batch = torch.as_tensor(img_window, dtype=torch.float32).to(device)
        label_tensor = torch.as_tensor(label, dtype=torch.float32).to(device)
        
        # Forward step and calculate loss
        with autocast():
          output = model(window_batch)
          val_loss = -torch.sum(label_tensor.mul(torch.log(output+eps)))/(output.shape[0]*output.shape[1]*output.shape[2])
        
        loss = train_loss.detach().cpu().numpy()
        val_loss_total += loss
        vepoch.set_postfix(loss=loss)

    # Log validation losses
    tqdm.write(f"Validation\t epoch: {epoch}/{num_epochs}\t loss: {val_loss_total/batch_num}")
    val_loss_log.append(val_loss_total/batch_num)

  #Save model  
  tqdm.write("Saving model")
  state = {'model':model.state_dict(), 'optimizer':optimizer.state_dict(), 'epoch':epoch, 'train_loss_log':train_loss_log, 'val_loss_log':val_loss_log}
  savePath = f"{savePath_base}/New_loader/TrackNet_{epoch}.pth"
  torch.save(state, savePath)
  model_saved = True

  plt.clf()
  plt.figure(dpi=300)
  plt.plot(range(1, epoch+1),train_loss_log,label='Train loss')
  plt.plot(range(1, epoch+1),val_loss_log,label='Validation loss')
  plt.legend()
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.title('Training loss')
  plt.show()


## Data Cleanup

In [ ]:
def reject_outliers(data, m = 2.):
  out = np.copy(data)
  d = np.abs(data - np.median(data))
  mdev = np.median(d)
  s = d/mdev if mdev else 0.
  outlier_ids = np.nonzero(s>m)
  for id in outlier_ids:
    out[id] = -1
  return out

def interp(data):
  for row in range(data.shape[0]):
    # Prevent end of array error
    if row == data.shape[0]-1:
      continue
    if data[row+1] == -1:
      shift = 1
      end_of_data = False
      while data[row+shift] == -1:
        shift += 1
        if row+shift == data.shape[0]:
          end_of_data = True
          break
      
      # Repeat last seen prediction if there isnt a prediction for last frames
      change = 0 if end_of_data else data[row+shift] - data[row]

      prev = data[row]
      for i in range(1,shift):
       data[row+i] = prev+change/shift
       prev = data[row+i]
  return data

def smooth(x,window_len=11,window='hanning'):
  s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
  if window == 'flat': #moving average
      w=np.ones(window_len,'d')
  else:
      w=eval('np.'+window+'(window_len)')

  y=np.convolve(w/w.sum(),s,mode='valid')
  return y

def refine_detections(data, repeats=2, window_len=11):

  for i in range(repeats):
    out = np.copy(data)
    # Reject the outliers
    rejection_sz = 30
    for i in range(0, data.shape[0], rejection_sz//2):
      slice = data[i:i+rejection_sz-1]
      rejected = reject_outliers(slice)
      out[i:i+rejection_sz-1] = rejected
    
    # Interpolate between rejections
    data = interp(out)
    data[:2] = data[3]
    unsmooth = data

  unfiltered = data
  data = ndimage.median_filter(data, size=20)
  data = savgol_filter(data, 51, 3)
  minima = argrelextrema(data, np.less)[0]
  maxima = argrelextrema(data, np.greater)[0]

  return data, minima, maxima, unfiltered

def clean_extrema(extrema):
  out = []
  i = 0
  while i < len(extrema):
    if i+1 == len(extrema):
      i += 1
      continue
    cur = extrema[i]
    next = extrema[i+1]
    close = [cur]
    while next - cur <= 10:
      close.append(next)
      i += 1
      if i+1 >= len(extrema):
        break
      next = extrema[i+1]
    i += 1
    if len(close) == 1:
      continue
    out.append(close)

  for sub in out:
    avr = sum(sub) // len(sub)
    id = extrema.index(sub[0])
    for num in sub:
      extrema.remove(num)
    extrema.insert(id, avr)
  return extrema

## Demo Videos

### Localisation

In [8]:
transform = albumentations.Compose([
  albumentations.Resize(height=TN_height, width=TN_width, interpolation=1, always_apply=True, p=1),
])

class DemoDataset(Dataset):
  def __init__(self):
    self.window_size = 3
    self.window_paths = []
    self.jpeg_reader = TurboJPEG()
    self.img_paths = sorted(glob.glob('./tmp/*.jpg'))
    
    for frame_no, _ in enumerate(self.img_paths):
      if not (self.window_size-2 < frame_no <= len(self.img_paths)-2):
        continue
      
      # Get frame window image paths
      window_path = []
      window_path.append(self.img_paths[frame_no-1])
      window_path.append(self.img_paths[frame_no])
      window_path.append(self.img_paths[frame_no+1])
      self.window_paths.append(window_path)

  def loader(self, path):
    in_file = open(path, 'rb')
    image = self.jpeg_reader.decode(in_file.read(), 0)
    image = transform(**{"image": image})
    test = image['image']
    image = torch.from_numpy(image['image'])
    return image, test

  def __getitem__(self, index):
    img_1, test_1 = self.loader(self.window_paths[index][0])
    img_2, test_2 = self.loader(self.window_paths[index][1])
    img_3, test_3 = self.loader(self.window_paths[index][2])

    test = np.concatenate((test_1, test_2, test_3),axis=2)
    test = np.rollaxis(test, 2, 0)
    test = torch.from_numpy(test)
    img_window = test

    centre_img = cv.imread(self.window_paths[index][1])
    centre_img = cv.resize(centre_img, (TN_width, TN_height))
    centre_img.astype(np.float32)

    return img_window, centre_img

  def __len__(self):
    return len(self.window_paths)

# Extracts all the frames from a video and saves them
def preprocess(video_path):
  cap = cv.VideoCapture(video_path)
  ret, frame = cap.read()
  count=0
  while(ret):
    frame = cv.resize(frame, (1280,720), interpolation= cv.INTER_LINEAR)
    cv.imwrite(f"tmp/{count:04d}.jpg", frame)
    ret, frame = cap.read()
    count += 1
    if count %200 == 0:
      print(f"Frame {count}")
  cap.release

def predict(video_path, preprocessing=True):
  if preprocessing:
    print("Preprocessing")
    preprocess(video_path)
  print("Loading demo dataset")
  demo_loader = DataLoader(DemoDataset(), batch_size=1,shuffle=False,num_workers=num_workers,pin_memory=True,drop_last=True)
  colors = [(i, i, i) for i in range(0, 256)]

  print("Making Predictions")
  preds = []
  model.eval()
  with torch.no_grad():
    with tqdm(demo_loader, unit="batch") as depoch:
      for data_batch in depoch:
        depoch.set_description(f"Demo")
        # Read in demo batch
        img_window, centre_img = data_batch
        window_batch = torch.as_tensor(img_window,dtype=torch.float32).to(device)
        window_batch=window_batch.view(-1,9,360,640).to(device)

        # Forward step
        output = model(window_batch)
        output_re = np.array(output.detach().tolist())
        output_re = output_re.reshape((TN_height, TN_width, 256)).argmax(axis=2)
        output_re = output_re.astype(np.uint8)  

        heatmap  = cv.resize(output_re, (1280,720))
        ret,heatmap = cv.threshold(heatmap,127,255,cv.THRESH_BINARY)
        circles = cv.HoughCircles(heatmap, cv.HOUGH_GRADIENT,dp=1,minDist=1,param1=50,param2=2,minRadius=2,maxRadius=7)

        if circles is not None and len(circles) == 1:
          x = int(circles[0][0][0])
          y = int(circles[0][0][1])
          preds.append([x, y])
          tqdm.write(f"{x}, {y}")
        else:
          preds.append([-1, -1])
          tqdm.write(f"{-1}, {-1}")
    out = np.array(preds)
    return out

In [ ]:
model = TrackNet(inchannel=9).to(device)
savePath = f"{savePath_base}/New_loader/TrackNet_15.pth"
checkpoint = torch.load(savePath)
model.load_state_dict(checkpoint['model'])
preds = predict("./Videos/test.mp4", preprocessing=False)
np.savetxt('preds.txt', preds, fmt='%d')

### Clean up detections and predict events

In [ ]:
preds = np.loadtxt('preds.txt', dtype="int")
x, y = preds[:,0], preds[:,1]

i = 0
while x[i] == -1:
  i += 1
x[:i] = x[i]

i = 0
while y[i] == -1:
  i += 1
y[:i] = y[i]

plt.clf()
plt.title('X')
plt.plot(range(0,x.shape[0]), x)
plt.show()

plt.clf()
plt.title('Y')
plt.plot(range(0,y.shape[0]), y)
plt.show()

x_detections, x_minima, x_maxima = refine_detections(x, repeats=3)
y_detections, y_minima, y_maxima = refine_detections(y, repeats=3)

x_extrema = [*x_minima, *x_maxima]
x_extrema.sort()
x_extrema = clean_extrema(x_extrema)

y_maxima.sort()
y_maxima = clean_extrema(y_maxima)

plt.clf()
plt.plot(x_detections, label="Smoothed")
plt.plot(x, label="Base")
for extreme in x_extrema:
 plt.axvline(x = extreme, color = 'b',)
plt.title('X')
plt.legend()
plt.show()

plt.clf()
plt.plot(y_detections, label="Smoothed")
plt.plot(y, label="Base")
for max in y_maxima:
 plt.axvline(x = max, color = 'b',)
plt.title('Y')
plt.legend()
plt.show()

### Output Video

In [ ]:
input_video_path = './Videos/corner_rally.mp4'
output_video_path = './Results/corner_events_50.mp4'

cap = cv.VideoCapture(input_video_path)
fps = int(cap.get(cv.CAP_PROP_FPS))
output_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
output_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter(output_video_path,fourcc, fps, (output_width,output_height))

frame_no = 0
while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == True:
    cv.circle(frame,(int(x_detections[frame_no]),int(y_detections[frame_no])), 8, (0, 0, 255), 2)
    cv.circle(frame,(int(x[frame_no]),int(y[frame_no])), 8, (0, 255, 0), 2)
    if frame_no in x_extrema:
      cv.putText(frame, f"Hit - frame {frame_no}", (50, 100), cv.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0), 2)
    if frame_no in y_maxima:
      cv.putText(frame, f"Bounce - frame {frame_no}", (50, 150), cv.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)
    frame_no += 1
    out.write(frame)

    if (frame_no%50) == 0:
      print(frame_no)

  # Break the loop
  else:
    break

cap.release()
out.release()

## Evaluation

In [ ]:
def metrics(pre, gt, name):
  rmse = np.sqrt(np.mean((gt[:, :2]-pre)**2))
  mae_x = np.mean(np.abs((gt[:, 0]-pre[:, 0])))
  mae_y = np.mean(np.abs((gt[:, 1]-pre[:, 1])))
  medae_x = np.median(np.abs((gt[:, 0]-pre[:, 0])))
  medae_y = np.median(np.abs((gt[:, 1]-pre[:, 1])))

  print(f"{name}:")
  print(f"\tRMSE : {rmse:.3f}")
  print(f"\tMAE X : {mae_x:.3f}\t MAE Y : {mae_y:.3f}")
  print(f"\tMedAE X : {medae_x:.3f}\t MAE Y : {medae_y:.3f}")

def get_dists(pre, gt):
  gt = np.array(gt, dtype=np.float64)
  dists = np.linalg.norm(gt[:, :2] - pre, axis=1)
  dists[dists == 0] = 0.001
  log_dists = np.log10(dists)
  return log_dists

### Localisation Performance

#### TrackNet

In [ ]:
# Load predictions
data = np.loadtxt("./Results/01.txt")
for i in range(2, 31):
  data = np.append(data, np.loadtxt(f"./Results/{i:02}.txt"), axis=0)
print(data.shape)

# Load groundtruth
label = pd.read_csv("./Dataset/test/01/Clip6/Label.csv")
label = label.fillna(-1)
label = label.to_numpy()
label = label[:, 2:]
gt = label

base_dir = "./Dataset/test"
games = os.listdir(base_dir)
games.remove("groundtruth")
games.remove("01")
games.sort()

for game in games:
  clips = os.listdir(f"{base_dir}/{game}")
  for clip in clips:
    label_path = f"{base_dir}/{game}/{clip}/Label.csv"
    label = pd.read_csv(label_path)
    label = label.fillna(-1)
    label = label.to_numpy()
    label = label[:, 2:]
    gt = np.append(gt, label, axis=0)
print(gt.shape)

In [ ]:
# Split predictions and groundtruth into court types
hard_pre, hard_gt = data[:6000], gt[:6000]
grass_pre, grass_gt = data[6000:7500], gt[6000:7500]
clay_pre, clay_gt = data[7500:], gt[7500:]

metrics(data, gt, "All Courts")
metrics(hard_pre, hard_gt, "Hard Court")
metrics(grass_pre, grass_gt, "Grass Court")
metrics(clay_pre, clay_gt, "Clay Court")

all_dists_log = get_dists(data, gt)
hard_dists_log = get_dists(hard_pre, hard_gt)
grass_dists_log = get_dists(grass_pre, grass_gt)
clay_dists_log = get_dists(clay_pre, clay_gt)

plt.clf()
plt.figure(dpi=300)
plt.title(f'Differences in Predicted Location (TrackNet)')
b, bins, p = plt.hist(all_dists_log, bins=np.linspace(0, 3, 30), alpha=0.7, label="All Courts")
vals, bins, patches = plt.hist([hard_dists_log, grass_dists_log, clay_dists_log], bins=np.linspace(0, 3, 30), label=["Hard Courts", "Grass Courts", "Clay Courts"])
all_samples = np.sum(b)
hard_samples = np.sum(vals[0])
grass_samples = np.sum(vals[1])
clay_samples = np.sum(vals[2])

# Normalise the histogram
for i in range(vals.shape[1]):
  # Change all court values
  p[i].set_height(p[i].get_height()/all_samples)
  b[i] /= all_samples

  patches[0][i].set_height(patches[0][i].get_height()/hard_samples)
  vals[0][i] /= hard_samples

  patches[1][i].set_height(patches[1][i].get_height()/grass_samples)
  vals[1][i] /= grass_samples

  patches[2][i].set_height(patches[2][i].get_height()/clay_samples)
  vals[2][i] /= clay_samples

plt.legend()
labels = ['1', '10', '100', '1000']
plt.ylim([0, 0.3])
plt.xticks(np.arange(0, 4, 1), labels)
plt.xlabel('Distance (pixels)')
plt.ylabel('Ratio')
plt.savefig(f"./Results/base_court_diffs.png")
plt.show()

#### TrackNet*

In [ ]:
# Load predictions
games = os.listdir(base_dir)
games.remove("groundtruth")
games.sort()
data_refined = []

preds = np.loadtxt(f"./Results/01.txt")
x, y = preds[:,0], preds[:,1]

i = 0
while x[i] == -1:
  i += 1
x[:i] = x[i]

i = 0
while y[i] == -1:
  i += 1
y[:i] = y[i]

_, _, _, x_unfiltered = refine_detections(x, repeats=3)
_, _, _, y_unfiltered = refine_detections(y, repeats=3)
x_unfiltered.shape = (300,1)
y_unfiltered.shape = (300,1)
data_refined = np.hstack((x_unfiltered, y_unfiltered))
data_refined = data_refined.astype(int)

for game in range(2, 31):
  # Get predictions
  preds = np.loadtxt(f"{game:02}.txt")
  x, y = preds[:,0], preds[:,1]

  i = 0
  while x[i] == -1:
    i += 1
  x[:i] = x[i]

  i = 0
  while y[i] == -1:
    i += 1
  y[:i] = y[i]
  
  _, _, _, x_unfiltered = refine_detections(x, repeats=3)
  _, _, _, y_unfiltered = refine_detections(y, repeats=3)
  x_unfiltered.shape = (300,1)
  y_unfiltered.shape = (300,1)
  xy = np.hstack((x_unfiltered, y_unfiltered))
  xy = xy.astype(int)
  data_refined = np.append(data_refined, xy, axis=0)

In [ ]:
# Split predictions and groundtruth into court types
hard_pre_ref, hard_gt = data_refined[:6000], gt[:6000]
grass_pre_ref, grass_gt = data_refined[6000:7500], gt[6000:7500]
clay_pre_ref, clay_gt = data_refined[7500:], gt[7500:]

metrics(data_refined, gt, "All Courts")
metrics(hard_pre_ref, hard_gt, "Hard Court")
metrics(grass_pre_ref, grass_gt, "Grass Court")
metrics(clay_pre_ref, clay_gt, "Clay Court")

all_dists_log = get_dists(data_refined, gt)
hard_dists_log = get_dists(hard_pre_ref, hard_gt)
grass_dists_log = get_dists(grass_pre_ref, grass_gt)
clay_dists_log = get_dists(clay_pre_ref, clay_gt)

plt.clf()
plt.figure(dpi=300)
plt.title(f'Differences in Predicted Location (TrackNet*)')
b, bins, p = plt.hist(all_dists_log, bins=np.linspace(0, 3, 30), alpha=0.7, label="All Courts")
vals, bins, patches = plt.hist([hard_dists_log, grass_dists_log, clay_dists_log], bins=np.linspace(0, 3, 30), label=["Hard Courts", "Grass Courts", "Clay Courts"])
all_samples = np.sum(b)
hard_samples = np.sum(vals[0])
grass_samples = np.sum(vals[1])
clay_samples = np.sum(vals[2])

# Normalise the histogram
for i in range(vals.shape[1]):
  # Change all court values
  p[i].set_height(p[i].get_height()/all_samples)
  b[i] /= all_samples

  patches[0][i].set_height(patches[0][i].get_height()/hard_samples)
  vals[0][i] /= hard_samples

  patches[1][i].set_height(patches[1][i].get_height()/grass_samples)
  vals[1][i] /= grass_samples

  patches[2][i].set_height(patches[2][i].get_height()/clay_samples)
  vals[2][i] /= clay_samples

plt.legend()
labels = ['1', '10', '100', '1000']
plt.ylim([0, 0.3])
plt.xticks(np.arange(0, 4, 1), labels)
plt.xlabel('Distance (pixels)')
plt.ylabel('Ratio')
plt.savefig(f"./Results/processed_court_diffs.png")
plt.show()


### Event Performance

In [ ]:
base_dir = "./Dataset/test"
games = os.listdir(base_dir)
games.remove("groundtruth")
games.sort()

eve = 0
pre_3 = 0
pre_5 = 0
pre_10 = 0
pre_15 = 0
pre_30 = 0

for game in range(1, 31):
  # Get predictions
  preds = np.loadtxt(f"./Results/{game:02}.txt")
  x = preds[:,0]
  y = preds[:,1]

  i = 0
  while x[i] == -1:
    i += 1
  x[:i] = x[i]

  i = 0
  while y[i] == -1:
    i += 1
  y[:i] = y[i]

  # Load groundtruth
  clips = os.listdir(f"{base_dir}/{game:02}")
  gt = None
  for clip in clips:
    label_path = f"{base_dir}/{game:02}/{clip}/Label.csv"
    label = pd.read_csv(label_path)
    label = label.fillna(0)
    label = label.to_numpy()
    label = label[:, 4:]
    if gt is not None:
      gt = np.append(gt, label, axis=0)
    else:
      gt = label
  
  x_detections, x_minima, x_maxima, _ = refine_detections(x, repeats=3)
  y_detections, y_minima, y_maxima, _ = refine_detections(y, repeats=3)

  x_extrema = [*x_minima, *x_maxima]
  x_extrema.sort()
  x_extrema = clean_extrema(x_extrema)

  y_maxima = [*y_maxima]
  y_maxima.sort()
  y_maxima = clean_extrema(y_maxima)

  hit_ids = np.where(np.all(gt==[1.],axis=1))[0]
  bounce_ids = np.where(np.all(gt==[2.],axis=1))[0]
  event_num = 0
  pre_num = 0

  for id in hit_ids:
    eve += 1
    for pre in x_extrema:
      if id-3 <= pre <= id+3:
        pre_3 += 1
      if id-5 <= pre <= id+5:
        pre_5 += 1
      if id-10 <= pre <= id+10:
        pre_10 += 1
      if id-15 <= pre <= id+15:
        pre_15 += 1
      if id-30 <= pre <= id+30:
        pre_30 += 1

  for id in bounce_ids:
    eve += 1
    for pre in y_maxima:
      if id-3 <= pre <= id+3:
        pre_3 += 1
      if id-5 <= pre <= id+5:
        pre_5 += 1
      if id-10 <= pre <= id+10:
        pre_10 += 1
      if id-15 <= pre <= id+15:
        pre_15 += 1
      if id-30 <= pre <= id+30:
        pre_30 += 1

print(f"3 frame window : {pre_3}/{eve} = {pre_3/eve:.3f}")
print(f"5 frame window : {pre_5}/{eve} = {pre_5/eve:.3f}")
print(f"10 frame window : {pre_10}/{eve} = {pre_10/eve:.3f}")
print(f"15 frame window : {pre_15}/{eve} = {pre_15/eve:.3f}")
print(f"30 frame window : {pre_30}/{eve} = {pre_30/eve:.3f}")